In [1]:
!nvidia-smi

Tue Apr 16 17:39:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [5]:
!pip install spacy_cleaner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 16.4 MB/s eta 0:00:00:00:0100:01


In [6]:
import spacy, re
import spacy_cleaner
from spacy_cleaner.processing import removers, mutators

In [7]:
%%time
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

CPU times: user 34.5 ms, sys: 11.9 ms, total: 46.5 ms
Wall time: 85.4 ms


In [8]:
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [9]:
len(train)

7613

In [10]:
max(train["id"])

10873

In [11]:
train["keyword"].value_counts()

keyword
fatalities               45
armageddon               42
deluge                   42
harm                     41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: count, Length: 221, dtype: int64

In [12]:
train["location"].value_counts()

location
USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
Some pum pum             1
Paulton, England         1
SEA Server               1
todaysbigstock.com       1
???????? ?????????.      1
Name: count, Length: 3341, dtype: int64

In [13]:
train = train[train.columns[3:]]  # Removed Other columns
test = test[test.columns[3:]]

In [14]:
train

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [15]:
test

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan
...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,Storm in RI worse than last hurricane. My city...
3260,Green Line derailment in Chicago http://t.co/U...
3261,MEG issues Hazardous Weather Outlook (HWO) htt...


In [16]:
train.dtypes

text      object
target     int64
dtype: object

In [17]:
train["text"] = train["text"].str.lower()
test["text"] = test["text"].str.lower()

In [18]:
test.head(2)

,text
0,just happened a terrible car crash
1,"heard about #earthquake is different cities, s..."


In [19]:
train.head(2)

,text,target
0,our deeds are the reason of this #earthquake m...,1
1,forest fire near la ronge sask. canada,1


<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h2>SpaCy's pre-built nlp model</h2>
</div>

In [ ]:
# Setting up the pipeline to clean the data using spacy
nlp = spacy.load("en_core_web_sm")
nlp

<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h2>SpaCy_Cleaner Pipeline setup</h2>
</div>

In [ ]:
pipeline = spacy_cleaner.Cleaner(
    nlp,  # pre-built language model
    removers.remove_email_token,
    removers.remove_punctuation_token,
    removers.remove_stopword_token,
    removers.remove_url_token,
    removers.remove_number_token,
    mutators.mutate_lemma_token,
)

In [ ]:
train.head(5)

<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h2>Cleaning the data</h2>
</div>

In [ ]:
%%time
train["text"] = pipeline.clean(train["text"])

In [ ]:
train.head(5)  # Now we got the clean texts

In [ ]:
train["text"][100]

In [ ]:
train["text"][102]

## there were tags(usually a name , which doesnot have any effect in the output) in the sentences and some unwanted symbols which accompanies

eg .ir icemoon aftershock dubstep trapmusic dnb edm dance ices\x89 (I have seen this,but forget the which line it is ,😂)

In [ ]:
# let us remove the tags and unwanted symbols to make it more clean

SymbolRemover = lambda x: re.sub("[#=><\/.]", "", x)

TagRemover = lambda word: re.sub("@\w+", "", word)

train["text"] = train["text"].apply(SymbolRemover)

train["text"] = train["text"].apply(TagRemover)

In [ ]:
train["text"][102]

In [ ]:
train["text"][100]

In [ ]:
!ls

<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h3>Using the pre-build model from Hugging face(Transformers) for Classification</h3>
</div>

In [ ]:
# Let's write the logic for the Model in pytorch
from transformers import (
    AutoTokenizer,
    DataCollator,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_metric

# !pip install --upgrade transformers
import os

os.environ["WANDB_DISABLED"] = "True"

<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h2>Using the "distilbert/distilbert-base-uncased" from transformers</h2>
    <a href=https://huggingface.co/transformers/v3.0.2/model_doc/auto.html#>Click to Refer for further<a>
</div>

In [ ]:
model = "distilbert/distilbert-base-uncased"
learning_rate = 2e-5
epochs = 8

In [ ]:
train_split, valid_split = train_test_split(
    train, test_size=0.2
)  # sppliting the data for train and valid

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model
)  # Maping the TOkenizer for the model classification
train_ds = Dataset.from_pandas(train_split)
valid_ds = Dataset.from_pandas(valid_split)
test_ds = Dataset.from_pandas(test)


def process_token(example, tokenizer=tokenizer):
    return tokenizer(example["text"])


tokenized_train = train_ds.map(process_token)
tokenized_valid = valid_ds.map(process_token)
tokenized_test = test_ds.map(process_token)

In [ ]:
tokenized_train[0]

In [ ]:
tokenized_test[0]

In [ ]:
columns_to_remove = ["__index_level_0__"]
train_dataset = tokenized_train.remove_columns(columns_to_remove)
valid_dataset = tokenized_valid.remove_columns(columns_to_remove)
test_dataset = tokenized_test

In [ ]:
train_dataset[0]

In [ ]:
# valid_dataset[0]

In [ ]:
train_dataset = train_dataset.rename_column("target", "label")
valid_dataset = valid_dataset.rename_column("target", "label")

In [ ]:
train_dataset[0]

In [ ]:
test_dataset[0]

In [ ]:
def compute_metrics(eval_pred):
    load_acc = load_metric("accuracy")
    load_f1 = load_metric("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = load_acc.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    print("Accuracy:" + acc, "F1:" + f1)
    return {"acc": acc, "f1": f1}

In [ ]:
# train_dataset

<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h2>Training the DistiltBert model</h2>
</div>

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    per_device_train_batch_size=16,
    weight_decay=0.01,
    output_dir="/kaggle/working/distilbert",
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.args

In [ ]:
trainer.train()

<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h2>Getting the Predictions</h2>
</div>

In [ ]:
predictions = trainer.predict(test_dataset=test_dataset)

In [ ]:
predictions.predictions[0]

In [ ]:
predictions_Strength_Class = np.argmax(
    predictions.predictions, axis=1
)  # Return the indexof the max value class

In [ ]:
predictions_Strength_Class  # Shows the label of our texts/tweets

<div style="background-image: url('https://media.istockphoto.com/id/1341408852/video/colored-smoke-on-a-dark-background-blue-and-red-light-with-smoke.jpg?s=640x640&k=20&c=v2DQUY8IVbli_6FH_9KAs6YWRXlDdYiBJHfp7JFh7NY='); background-size: cover; background-position: center; border-radius: 10px; padding: 20px; color: #ffffff;">
    <h2>Making the Submission File</h2>
</div>

In [ ]:
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission["target"] = predictions_Strength_Class
submission = submission.set_index("id", drop=True)
submission.to_csv("/kaggle/working/submission.csv")

In [ ]:
submission.head(5)